In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time
from datetime import timedelta
import math
from tensorflow.examples.tutorials.mnist import input_data
import os
from PIL import Image 
from scipy.ndimage import filters
from scipy import ndimage

os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
#import opencv
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.05))
def new_biases(length):
    return tf.Variable(tf.constant(0.1,shape=length))
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
def max_pool_2x2(inputx):
    return tf.nn.max_pool(inputx,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

#import data
data = input_data.read_data_sets("/data/MNIST_data",one_hot=True)
#one_hot means [0 0 1 0 0 0 0 0 0 0] stands for 2

print("Size of:")
print("--Tranining-set:\t\t{}".format(len(data.train.labels)))
print("--Testing-set:\t\t{}".format(len(data.test.labels)))
print("--Validation-set:\t\t".format(len(data.validation.labels)))
data.test.cls = np.argmax(data.test.labels,axis=1)
image = data.test.images[50,:]
image_one = image.reshape((28,28))*255

x = tf.placeholder("float",shape=[None,784],name='x')
x_image = tf.reshape(x,[-1,28,28,1])

y_true = tf.placeholder("float",shape=[None,10],name='y_true')
y_true_cls = tf.argmax(y_true,axis=1)

#Conv 1
layer_conv1 = {"weights":new_weights([5,5,1,32]),
              "biases":new_biases([32])}
h_conv1 = tf.nn.relu(conv2d(x_image,layer_conv1["weights"])+layer_conv1["biases"])
h_pool1 = max_pool_2x2(h_conv1)

#Conv2
layer_conv2 = {"weights":new_weights([5,5,32,64]),
              "biases":new_biases([64])}
h_conv2 = tf.nn.relu(conv2d(h_pool1,layer_conv2["weights"])+layer_conv2["biases"])
h_pool2 = max_pool_2x2(h_conv2)

#Full.connected layer 1
fcl_layer = {"weights":new_weights([7*7*64,1024]),
            "biases":new_biases([1024])}
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
h_fcl = tf.nn.relu(tf.matmul(h_pool2_flat,fcl_layer["weights"])+fcl_layer["biases"])

#Droupout layer
keep_prob = tf.placeholder("float")
h_fcl_drop = tf.nn.dropout(h_fcl,keep_prob)

#Full - connected layer 2
fc2_layer = {"weights":new_weights([1024,10]),
            "biases":new_biases([10])}

#Predicted class
y_pred = tf.nn.softmax(tf.matmul(h_fcl_drop,fc2_layer["weights"])+fc2_layer["biases"])
y_pred_cls = tf.argmax(y_pred,axis=1)

#cost function to be optimized 
cross_entropy = -tf.reduce_mean(y_true*tf.log(y_pred))
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cross_entropy)

# performance measures
correct_prediction = tf.equal(y_pred_cls,y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction,'float'))
sess2 = tf.InteractiveSession()

#optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.2).minimize(accuracy)
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    train_batch_size = 50
    def optimize(num_iterations):
        total_iterations=0
        start_time = time.time()
        for i in range(total_iterations,total_iterations+num_iterations):
            x_batch,y_true_batch = data.train.next_batch(train_batch_size)
            feed_dict_train_op = {x:x_batch,y_true:y_true_batch,keep_prob:0.5}
            feed_dict_trian = {x:x_batch,y_true:y_true_batch,keep_prob:1.0}
            cost = sess.run(optimizer,feed_dict=feed_dict_train_op)
            cost2 = sess.run(cross_entropy,feed_dict=feed_dict_train_op)
            
            if cost2:
                print("cost:%f" % cost2)
            if i&100==0:
                acc = sess.run(accuracy,feed_dict=feed_dict_trian)
                msg = "Optimizetion Iteration:{0:>6},Training Accuracy: {1:>6.1%}"
                print(msg.format(i+1,acc))
        
        total_iterations += num_iterations
        end_time = time.time()
        time_dif = end_time-start_time
        print("Time usage:"+str(timedelta(seconds=int(round(time_dif)))))
    test_batch_size = 256
    def print_test_accuracy():
        num_test = len(data.test.images)
        cls_pred = np.zeros(shape=num_test,dtype=np.int)
        i= 0
        while i < num_test:
            j = min(i+test_batch_size,num_test)
            images = data.test.images[i:j,:]
            labels = data.test.labels[i:j,:]
            feed_dict={x:images,y_true:labels,keep_prob:1.0}
            cls_pred[i:j] = sess.run(y_pred_cls,feed_dict=feed_dict)
            i = j
        cls_true  = data.test.cls
        correct = (cls_true==cls_pred)
        correct_sum = correct.sum()
        acc = float(correct_sum) / num_test
        msg = "Accuracy on Test-Set:{0:.1%} ({1}/{2})"
        print(msg.format(acc,correct_sum,num_test))
        
    optimize(500)
    print_test_accuracy()

Extracting /data/MNIST_data/train-images-idx3-ubyte.gz
Extracting /data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /data/MNIST_data/t10k-labels-idx1-ubyte.gz
Size of:
--Tranining-set:		55000
--Testing-set:		10000
--Validation-set:		


/usr/local/lib/python3.5/dist-packages/tensorflow/python/client/session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


cost:0.243675
Optimizetion Iteration:     1,Training Accuracy:  22.0%
cost:0.247835
Optimizetion Iteration:     2,Training Accuracy:  22.0%
cost:0.218030
Optimizetion Iteration:     3,Training Accuracy:  28.0%
cost:0.236617
Optimizetion Iteration:     4,Training Accuracy:  28.0%
cost:0.242512
cost:0.237807
cost:0.248112
cost:0.222205
cost:0.206047
Optimizetion Iteration:     9,Training Accuracy:  36.0%
cost:0.230523
Optimizetion Iteration:    10,Training Accuracy:  20.0%
cost:0.232549
Optimizetion Iteration:    11,Training Accuracy:  32.0%
cost:0.231077
Optimizetion Iteration:    12,Training Accuracy:  24.0%
cost:0.215651
cost:0.215026
cost:0.220597
cost:0.204358
cost:0.216777
Optimizetion Iteration:    17,Training Accuracy:  36.0%
cost:0.193772
Optimizetion Iteration:    18,Training Accuracy:  44.0%
cost:0.205897
Optimizetion Iteration:    19,Training Accuracy:  32.0%
cost:0.220163
Optimizetion Iteration:    20,Training Accuracy:  26.0%
cost:0.211767
cost:0.202031
cost:0.200927
cost:0

cost:0.027137
cost:0.040653
cost:0.032421
cost:0.021858
Optimizetion Iteration:   393,Training Accuracy:  96.0%
cost:0.050172
Optimizetion Iteration:   394,Training Accuracy:  86.0%
cost:0.034232
Optimizetion Iteration:   395,Training Accuracy:  92.0%
cost:0.025009
Optimizetion Iteration:   396,Training Accuracy:  94.0%
cost:0.024736
cost:0.027337
cost:0.036017
cost:0.033554
cost:0.024217
Optimizetion Iteration:   401,Training Accuracy:  94.0%
cost:0.032355
Optimizetion Iteration:   402,Training Accuracy:  92.0%
cost:0.016827
Optimizetion Iteration:   403,Training Accuracy:  98.0%
cost:0.023340
Optimizetion Iteration:   404,Training Accuracy:  96.0%
cost:0.018226
cost:0.015944
cost:0.032125
cost:0.018706
cost:0.016492
Optimizetion Iteration:   409,Training Accuracy:  96.0%
cost:0.038015
Optimizetion Iteration:   410,Training Accuracy:  88.0%
cost:0.028238
Optimizetion Iteration:   411,Training Accuracy:  92.0%
cost:0.025105
Optimizetion Iteration:   412,Training Accuracy:  96.0%
cost:0